In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from libpysal.graph import read_parquet
from shapely import unary_union
import momepy as mm

In [2]:


model_params = '_75_0_None_None_False'
buildings_dir = '/data/uscuni-ulce/processed_data/buildings/'
graph_dir = '/data/uscuni-ulce/processed_data/neigh_graphs/'
morph_dir = '/data/uscuni-ulce/processed_data/morphotopes/'

In [3]:
# region_id = 139196 # prague

region_id = 69333 



# region_id = 8707
# region_id = 5883 # freiburtg
# region_id = 86873
# region_id = 38679
# region_id = 55763
# region_id = 107131
# region_id = 99886

# region_id = 151676 # vilnius

# region_id= 66593

In [4]:

etcs = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/tessellations_chars_{region_id}.parquet')
tess = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/tessellations_chars_{region_id}.parquet')
buildings = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/buildings_chars_{region_id}.parquet')
streets = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/streets_chars_{region_id}.parquet')
nodes = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/nodes_chars_{region_id}.parquet')
primary = pd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/primary_chars_{region_id}.parquet')

In [ ]:
# streets.explore()

In [9]:
### add the connected structure characters
buildings_q1 = read_parquet(graph_dir + f"building_graph_{region_id}.parquet")
clusters = pd.read_parquet(f'{morph_dir}tessellation_labels_morphotopes_{region_id}_75_0_None_None_False.pq')

In [10]:
buildings = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/buildings_chars_{region_id}.parquet')
buildings['morph'] = clusters
buildings = buildings[buildings.morph.str.split('_').str[-1] != '-1']

In [11]:
# buildings = buildings.join(primary.drop(columns=primary.columns[~primary.columns.isin(streets.columns)]))

In [12]:
morph_primary = primary.groupby(clusters.morphotope_label).median()


In [13]:
morphs_to_check = ['849_437', '849_486', '849_530']

In [14]:
morph_primary.loc[morphs_to_check].style.background_gradient(axis=0, cmap="BuGn")

,sdbAre,sdbPer,sdbCoA,ssbCCo,ssbCor,ssbSqu,ssbERI,ssbElo,ssbCCM,ssbCCD,stbOri,mtbSWR,libNCo,ldbPWL,ltcBuA,mtbAli,mtbNDi,ltbIBD,stbCeA,stbSAl,sdsLen,sssLin,ldsMSL,ldsRea,ldsAre,sisBpM,sdsSPW,sdsSPO,sdsSWD,mtdDeg,lcdMes,linP3W,linP4W,linPDE,lcnClo,lddNDe,linWID,ldsCDL,xcnSCl,mtdMDi,sddAre,midRea,midAre,stcOri,sdcLAL,sdcAre,sscCCo,sscERI,mtcWNe,mdcAre,ltcWRB,sicCAR,stcSAl,ldkAre,ldkPer,lskCCo,lskERI,lskCWA,ltkOri,ltkWNB,likWBB,sdsAre,likWCe,mibCou,mibAre,mibLen,mibElo,mibERI,mibCCo,mibLAL,mibFR,mibSCo,micBAD,licBAD,misBAD,midBAD
morphotope_label,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
849_437,859.241907,160.023023,0.000000,0.372928,6.500000,0.537349,0.892025,0.436748,16.353140,5.096996,13.463040,0.000000,0.000000,191.824077,1.000000,1.692788,42.985776,44.363624,4.385041,2.736864,160.279569,0.999345,141.066275,40.000000,325849.946670,0.014791,37.550729,0.678571,2.723619,3.000000,0.062096,0.616667,0.173214,0.192308,0.000002,0.006401,0.011654,269.241291,0.000000,135.774085,17306.180477,7.000000,49321.808451,14.819522,137.098373,6685.951497,0.489169,0.967229,0.019503,61834.842413,0.000024,0.144241,3.363489,87015.401871,1446.048639,0.434799,0.859706,354.843976,27.061449,0.005947,0.144052,20464.764032,0.000149,1.000000,1262.130170,191.271685,0.343404,0.863177,0.306046,74.515440,5.787116,5.787116,918.999758,2294.365067,113.351450,7.442156
849_486,270.390609,73.599375,0.000000,0.489031,4.000000,0.315569,0.999560,0.514805,13.285515,0.050556,38.350533,0.000000,0.000000,92.626620,1.000000,1.015337,23.004441,30.993210,4.081491,2.865691,913.478904,0.996533,335.089914,253.000000,1177635.379728,0.062148,36.937070,0.917411,5.793012,3.000000,0.048780,0.791667,0.083333,0.125000,0.000000,0.003464,0.006199,1311.491326,0.000000,754.804345,263407.949778,81.000000,457167.974576,34.987521,79.805855,2317.068809,0.482116,0.982426,0.031781,27075.723908,0.000008,0.142837,6.747334,1282697.063445,10348.304584,0.336240,0.452199,2383.720505,38.490951,0.001643,0.153213,257918.023654,0.000253,1.000000,411.302682,91.684520,0.530516,0.999874,0.483515,34.880123,4.537217,4.537217,598.422528,1584.358859,753.772258,1468.646816
849_530,115.402701,44.910727,0.000000,0.514366,4.000000,0.485372,0.999773,0.522744,8.002523,0.039238,2.542587,0.000000,0.000000,48.878672,1.000000,2.361025,26.980348,31.387949,1.814443,1.390285,481.947861,0.999994,324.409365,250.000000,1701981.371189,0.069826,50.000000,1.000000,2.187555,3.000000,0.068966,0.581395,0.166667,0.232558,0.000001,0.003022,0.005213,1025.652356,0.076923,306.914028,127364.296664,47.000000,282247.485876,8.331863,98.204769,3464.072867,0.479405,0.987021,0.025488,43314.668052,0.000011,0.060025,6.119848,530310.416180,3985.008822,0.381886,0.765244,982.556967,23.237544,0.001757,0.159062,120634.260347,0.000168,1.000000,128.558489,47.936573,0.536681,1.002601,0.522347,17.611684,2.769253,2.769253,1069.251795,2841.582940,589.096235,1232.574771


In [15]:
from core.utils import used_keys
used_keys['ldsCDL']

'local cul-de-sac length of street network'

In [16]:
groups = buildings.groupby('morph')

In [17]:
def generate_ratio(group, buildings_q1):
    
    group_graph = buildings_q1.subgraph(group.index)
    connected_buildings = group.geometry.groupby(group_graph.component_labels).apply( lambda x: unary_union(x.values))
    libNCos = group.libNCo.median()
    connected_buildings = connected_buildings

    areas = connected_buildings.area
    elongation = mm.elongation(connected_buildings)
    fr = mm.facade_ratio(connected_buildings)
    thin = ((fr < 8)   & (elongation < .90))

    
    morph_fr_area_ratio = areas[thin].sum() / areas.sum()
    morph_fr_count_ratio = connected_buildings[thin].count() / connected_buildings.count()

   
    largest = areas > areas.median()
    largest_thin = thin & largest

    largest_morph_fr_area_ratio = areas[largest_thin].sum() / areas[largest].sum()

    limLPS = ((morph_fr_area_ratio > .4) | (libNCos > 0)).astype(int)
    
    return pd.Series({'limMFR': morph_fr_area_ratio,
                      'limMTC': morph_fr_count_ratio,
                      'limLMFR': largest_morph_fr_area_ratio,
                      'limLPS': limLPS
                     }
    )

In [18]:
res = groups.apply(generate_ratio, buildings_q1)

/tmp/ipykernel_826286/2076471594.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  res = groups.apply(generate_ratio, buildings_q1)


In [19]:
res

,limMFR,limMTC,limLMFR,limLPS
morph,,,,
1000_0,0.796800,0.900000,0.777366,1.0
1005_0,0.929558,0.904110,0.925263,1.0
1016_0,0.560153,0.851675,0.501304,1.0
1016_1,0.885192,0.866667,0.923389,1.0
1016_2,0.827632,0.811765,0.881292,1.0
...,...,...,...,...
979_0,0.280248,0.782051,0.243906,0.0
991_0,0.791131,0.886598,0.761129,1.0
991_1,0.934296,0.919708,0.942193,1.0


In [20]:
plotting = buildings.copy()

In [21]:
plotting['morph_fr_area_ratio'] = buildings.morph.map(res.limMFR.to_dict())
plotting['morph_fr_count_ratio'] = buildings.morph.map(res.limMTC.to_dict())
plotting['morph_micBAD'] = buildings.morph.map(res.limLMFR.to_dict())
plotting['limLPS'] = buildings.morph.map(res.limLPS.to_dict())

In [22]:
import lonboard
layer = lonboard.SolidPolygonLayer.from_geopandas(plotting, opacity=.7)

from sidecar import Sidecar
sc = Sidecar(title=f'Final Clusters')
m = lonboard.Map(layer, basemap_style=lonboard.basemap.CartoBasemap.DarkMatter)
with sc:
    display(m)

/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")


In [23]:
# res['limLPS'] = ((res['limMFR'] > .4) | (buildings['libNCo'] > 0)).astype(int)

In [24]:
# thin = (buildings.morph.map((morph_primary.sdsLen > 450).astype(int).to_dict())).astype(int)

# thin = (buildings.morph.map((morph_primary.midBAD > 1000).astype(int).to_dict())).astype(int)


# thin = (buildings.morph.map((morph_primary.mibERI > .99).astype(int).to_dict())).astype(int)


In [25]:

from core.cluster_validation import get_color
colors = get_color(plotting.limLPS)
layer.get_fill_color = colors

In [21]:
# connected_buildings.explore(column=thin, categorical=True)

## Parallel processing

In [5]:
def morphotopes_to_etcs(region_id, etcs=True, model_params='_100_0_None_None_False'):

    if etcs:
        etcs = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/tessellations/tessellation_{region_id}.parquet')

    else:
        etcs = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/buildings/buildings_{region_id}.parquet')
        
    etcs['label'] = -1
    
    morphotopes = pd.read_parquet(f'/data/uscuni-ulce/processed_data/morphotopes/tessellation_labels_morphotopes_{region_id}{model_params}.pq')
    morphotopes.loc[:, 'morphotope_label'] =  morphotopes.values[:, 0]

    morph_dict = pd.Series(np.arange(np.unique(morphotopes.values).shape[0]),
                       np.unique(morphotopes.values))
    etcs.loc[morphotopes.index, 'label'] = morphotopes.map(lambda x: morph_dict.loc[x]).values
    etcs['morph'] = str(region_id) + '_' + '-1'
    etcs.loc[morphotopes.index, 'morph'] = str(region_id) + '_' + morphotopes.values
    return etcs


def generate_ratio(group, buildings_q1):
    
    group_graph = buildings_q1.subgraph(group.index)
    connected_buildings = group.geometry.groupby(group_graph.component_labels).apply( lambda x: unary_union(x.values))
    libNCos = group.libNCo.median()
    connected_buildings = connected_buildings

    areas = connected_buildings.area
    elongation = mm.elongation(connected_buildings)
    fr = mm.facade_ratio(connected_buildings)
    thin = ((fr < 8)   & (elongation < .90))

    
    morph_fr_area_ratio = areas[thin].sum() / areas.sum()
    morph_fr_count_ratio = connected_buildings[thin].count() / connected_buildings.count()

   
    largest = areas > areas.median()
    largest_thin = thin & largest

    largest_morph_fr_area_ratio = areas[largest_thin].sum() / areas[largest].sum()

    limLPS = ((morph_fr_area_ratio > .4) | (libNCos > 0)).astype(int)
    
    return pd.Series({'limMFR': morph_fr_area_ratio,
                      'limMTC': morph_fr_count_ratio,
                      'limLMFR': largest_morph_fr_area_ratio,
                      'limLPS': limLPS
                     }
    )

In [6]:
region_id = 99886 # vilnius
region_id = 69333 # prague


In [7]:
def add_morph_chars(region_id):
    
    etcs = morphotopes_to_etcs(region_id, model_params=model_params)
    buildings = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/buildings_chars_{region_id}.parquet')

    morphs = etcs[etcs.morph.str.split('_').str[-1] != '-1'].to_crs(epsg=3035)
    buildings = buildings.loc[morphs.index]
    buildings['morph'] = etcs['morph']
    buildings_q1 = read_parquet(graph_dir + f"building_graph_{region_id}.parquet")
    res = buildings.groupby('morph').apply(generate_ratio, buildings_q1)
        
    res.to_parquet(f'/data/uscuni-ulce/processed_data/morphotopes/morph_chars_{region_id}.pq')

In [8]:
regions_datadir = "/data/uscuni-ulce/"
region_hulls = gpd.read_parquet(
    regions_datadir + "regions/" + "cadastre_regions_hull.parquet"
)

In [9]:
# region_hulls = region_hulls.loc[[region_id]]

In [10]:
from joblib import Parallel, delayed
n_jobs = -1
new = Parallel(n_jobs=n_jobs)(
    delayed(add_morph_chars)(region_id) for region_id, _ in region_hulls.iterrows())

/tmp/ipykernel_826440/1632418541.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
/tmp/ipykernel_826440/1632418541.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
/tmp/ipykernel_826440/1632418541.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Eith